In [1]:
import time
from sim_worker_threads import Simulations, SimUtils

DATA_PATH = r".\data.min.js" # data.min.js
SIM_PATH = r".\simulator.js" # simulator.js
SECRET_PATH = r".\client_secret_XXXXXXXXXXXX.apps.googleusercontent.com.json" # google OAuth .json

MASS_SIM_SHEET_NAME = "Beast/Aether/Avian Guild War" # Cloud's sheet name
OPEN_SIM_SHEET_NAME = "Spellstone Open Mass Sim" # Chef's sheet name

MAX_THREADS = 3 # Number of CPU threads to use. Recommended: all available minus 1
NUM_SIMS = 1000 # Number of simulations per off-def pair
BGE = SimUtils.get_current_bges(DATA_PATH, add_GC=False) # Automatically read current BGEs from file
SIEGE = True # Whether the opponent has any tower. Default True (e.g. bounties)
TOWER = "501" # The opponent's tower type ID. Default "501" (e.g. bounties)
DELAY = 15 # Minimum time interval between sheet updates in seconds

In [2]:
sim = Simulations(SIM_PATH, MAX_THREADS, BGE, SIEGE, TOWER)
open_sim = SimUtils.SheetUtils(SECRET_PATH, OPEN_SIM_SHEET_NAME)
full_attackers, full_defenders = open_sim.get_hash_stash()

In [ ]:
# sim = Simulations(SIM_PATH, MAX_THREADS, BGE, SIEGE, TOWER)
# mass_sim = SimUtils.SheetUtils(SECRET_PATH, MASS_SIM_SHEET_NAME)
# full_attackers, full_defenders = mass_sim.get_cloud_hashes()

In [3]:
# infinite loop
try:
    while True:
        ## core block
        start_time = time.time()

        attackers, defenders, off_notes, def_notes = open_sim.get_open_sim_hashes()

        if len(attackers) > 0:
            off_results = sim.mass_sim(attackers, full_defenders, NUM_SIMS, include_top50=True)
            open_sim.update_open_sim_results(off_results.iloc[2:, :2].reindex(attackers).assign(**{"Notes/Names": off_notes}), "off", fifo=True)

        if len(defenders) > 0:
            def_results = sim.mass_sim(full_attackers, defenders, NUM_SIMS, include_top50=True)
            open_sim.update_open_sim_results(def_results.T.iloc[2:, :2].reindex(defenders).assign(**{"Notes/Names": def_notes}), "def", fifo=True)

        print(f"Execution time: {time.time() - start_time:.2f} seconds  ", end="\r")        

        # Wait until the next N-second interval
        current_time = time.localtime()
        seconds_until_next_interval = DELAY - (current_time.tm_sec % DELAY)
        time.sleep(seconds_until_next_interval)

except KeyboardInterrupt:
    print("Execution stopped by user     ")